In [160]:
import os
import pandas as pd
import numpy as np

from datetime import datetime, timedelta

from prophet import Prophet
from sklearn.metrics import mean_squared_error, mean_absolute_error

import plotly.express as px
import plotly.graph_objects as go
import plotly.io as pio

import os


## Import dataset

In [161]:
path = "./data/01aJourneyDataExtract10Jan16-23Jan16.csv"
dataset = pd.read_csv(path)
dataset.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,50754225,240,11834,10/01/2016 00:04,383.0,"Frith Street, Soho",10/01/2016 00:00,18,"Drury Lane, Covent Garden"
1,50754226,300,9648,10/01/2016 00:05,719.0,"Victoria Park Road, Hackney Central",10/01/2016 00:00,479,"Pott Street, Bethnal Green"
2,50754227,1200,10689,10/01/2016 00:20,272.0,"Baylis Road, Waterloo",10/01/2016 00:00,425,"Harrington Square 2, Camden Town"
3,50754228,780,8593,10/01/2016 00:14,471.0,"Hewison Street, Old Ford",10/01/2016 00:01,487,"Canton Street, Poplar"
4,50754229,600,8619,10/01/2016 00:11,399.0,"Brick Lane Market, Shoreditch",10/01/2016 00:01,501,"Cephas Street, Bethnal Green"


# Data cleaning

In [162]:
df = dataset
df[['End Date', 'Start Date']] = dataset[['End Date', 'Start Date']].apply(pd.to_datetime, dayfirst=True).apply(lambda x: x.dt.normalize())
df['Duration'] = pd.to_timedelta(df['Duration'], unit='m')

df.head()

,Rental Id,Duration,Bike Id,End Date,EndStation Id,EndStation Name,Start Date,StartStation Id,StartStation Name
0,50754225,0 days 04:00:00,11834,2016-01-10,383.0,"Frith Street, Soho",2016-01-10,18,"Drury Lane, Covent Garden"
1,50754226,0 days 05:00:00,9648,2016-01-10,719.0,"Victoria Park Road, Hackney Central",2016-01-10,479,"Pott Street, Bethnal Green"
2,50754227,0 days 20:00:00,10689,2016-01-10,272.0,"Baylis Road, Waterloo",2016-01-10,425,"Harrington Square 2, Camden Town"
3,50754228,0 days 13:00:00,8593,2016-01-10,471.0,"Hewison Street, Old Ford",2016-01-10,487,"Canton Street, Poplar"
4,50754229,0 days 10:00:00,8619,2016-01-10,399.0,"Brick Lane Market, Shoreditch",2016-01-10,501,"Cephas Street, Bethnal Green"


In [163]:
df_time = df[["Duration", "Start Date"]]
df_time.head()

,Duration,Start Date
0,0 days 04:00:00,2016-01-10
1,0 days 05:00:00,2016-01-10
2,0 days 20:00:00,2016-01-10
3,0 days 13:00:00,2016-01-10
4,0 days 10:00:00,2016-01-10


# Feature selection
Select duration and date

In [164]:
df_time = df_time.sample(n=10000, random_state=4)
df_time.head()

,Duration,Start Date
219517,0 days 16:00:00,2016-01-20
58640,1 days 02:00:00,2016-01-13
31071,0 days 13:00:00,2016-01-11
10158,1 days 01:00:00,2016-01-10
163578,1 days 07:00:00,2016-01-18


In [174]:
df_time = df_time.rename(columns={'Start Date': 'ds', 'Duration': 'y'})
df_time.head()

,y,ds
219517,0 days 16:00:00,2016-01-20
58640,1 days 02:00:00,2016-01-13
31071,0 days 13:00:00,2016-01-11
10158,1 days 01:00:00,2016-01-10
163578,1 days 07:00:00,2016-01-18


In [175]:
# Initialiser le modèle Prophet
model = Prophet()

# Ajuster le modèle sur les données préparées
model.fit(df_time)

# Faire des prévisions pour les prochaines périodes
future = model.make_future_dataframe(periods=30, freq='D')  # Prédictions pour les 30 prochains jours
forecast = model.predict(future)

# Afficher les prévisions
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].head()

AttributeError: 'Prophet' object has no attribute 'stan_backend'

In [ ]:
# Tracer les prévisions avec Prophet
fig = model.plot(forecast)

# Afficher le graphique
plt.show()

# Obsolete
-------------------

In [165]:
# morning = "09:00:00"
# evening = "17:00:00"
# morning_time = datetime.strptime(morning, "%H:%M:%S").time()
# evening_time = datetime.strptime(evening, "%H:%M:%S").time()
# print(morning_time, evening_time)

## Morning hours

In [166]:
# # Calcul de morning_start et morning_end
# morning_start = (datetime.combine(datetime.today(), morning_time) - timedelta(hours=1)).time()
# morning_end = (datetime.combine(datetime.today(), morning_time) + timedelta(hours=1)).time()

# df_morning = df[(df['Start Date'] >= morning_start) & (df['Start Date'] <= morning_end)]
# df_morning = df_morning.reset_index(drop=True)
# df_morning.head()

In [167]:
# # Elimination colones superflues
# df_morning = df_morning.drop(["Duration","Rental Id", "End Date","EndStation Id", "EndStation Name"], axis=1)
# df_morning.head()

In [168]:
# df_morning.columns

## Evening hours

In [169]:
# # Calcul de evening_start et evening_end
# evening_start = (datetime.combine(datetime.today(), evening_time) - timedelta(hours=1)).time()
# evening_end = (datetime.combine(datetime.today(), evening_time) + timedelta(hours=1)).time()

# df_evening = df[(df['Start Date'] >= evening_start) & (df['Start Date'] <= evening_end)]
# df_evening = df_evening.reset_index(drop=True)
# df_evening.head()

In [170]:
# # Elimination colones superflues
# df_evening = df_evening.drop(["Duration","Rental Id", "End Date","EndStation Id", "EndStation Name"], axis=1)
# df_evening.head()

In [171]:
# print(df_morning.dtypes)

# Machine Learning
## Morning

In [172]:
# X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)